In [1]:
#Try to install libraries if needed, can skip if not

In [2]:
!pip install sqlalchemy pymysql

Defaulting to user installation because normal site-packages is not writeable


In [3]:
from sqlalchemy import create_engine, text, inspect
import panel as pn
import pandas as pd
pn.extension("tabulator")

In [4]:
import sys
sys.path.append("../")
from src import mariadb_data

In [5]:
first_row_height = 400
table_names = mariadb_data.get_table_names()

columns_pn = pn.widgets.Tabulator(
    mariadb_data.get_table_schema(table_names[0]),
    disabled=True,
    show_index=False,
    height=first_row_height,
    layout="fit_data",
    hidden_columns=["default", "comment", "nullable", "autoincrement"],
    titles={
        "name": "Column Name",
        "type": "Column Type"
    }
)

table_data_pn = pn.widgets.Tabulator(
    mariadb_data.get_table_data(table_names[0], limit=100),
    disabled=True,
    height=500,
    sizing_mode='stretch_width',
)

table_names_df = pd.DataFrame(table_names, columns=["Table"])
table_names_pn = pn.widgets.Tabulator(
    value=table_names_df,
    disabled=True,
    show_index=False,
    selection=[0],
    selectable=1,
    height=first_row_height,
    #layout='fit_data',
    #width=500,
    #sizing_mode='stretch_width'
)
def on_table_name_click(event):
    table_name = event.value
    
    column_info = mariadb_data.get_table_schema(table_name)
    table_data = mariadb_data.get_table_data(table_name, limit=100)
    
    columns_pn.param.update(selection=[])
    columns_pn.param.update(value=column_info)
    table_data_pn.param.update(value=table_data)
    
table_names_pn.on_click(on_table_name_click)

gspec = pn.GridSpec(sizing_mode='stretch_both') #max_height=800)
gspec[0, 0] = pn.Column(pn.pane.Markdown("## Tables"), table_names_pn)
gspec[0, 1:3] = pn.Column(pn.pane.Markdown("## Columns"), columns_pn)
gspec[1, 0:3] =   pn.Column(pn.pane.Markdown("## Sample Data"), table_data_pn)

In [6]:
gspec

GridSpec(ncols=3, nrows=2, sizing_mode='stretch_both')
    [0] Column(sizing_mode='stretch_both')
        [0] Markdown(str)
        [1] Tabulator(disabled=True, height=400, selection=[0], show_index=False, value=              ...)
    [1] Column(sizing_mode='stretch_both')
        [0] Markdown(str)
        [1] Tabulator(disabled=True, height=400, hidden_columns=['default', 'comment', ...], layout='fit_data', show_index=False, titles={'name': 'Column Name', ...}, value=       name   ...)
    [2] Column(sizing_mode='stretch_both')
        [0] Markdown(str)
        [1] Tabulator(disabled=True, height=500, sizing_mode='stretch_width', value=    piece_id  ...)